In [1]:
import networkx as nx
import heapq as hq
from copy import deepcopy

import distanceclosure as dc

# For some reason pylance doesn't like these imports, but they work fine
import KnowledgeGraph as kg  # type: ignore
import ConceptNode as cn  # type: ignore
from MultiDistance import MultiDistance, multimin, sum_pareto_distance  # type: ignore

from multiprocessing import Pool
import mp_for_nb

import pickle

In [2]:
G0=nx.read_gpickle('Sample DDI networks/net-ddi-bnu-tau.gpickle')
G1=nx.read_gpickle('Sample DDI networks/net-ddi-cat-tau.gpickle')
G2=nx.read_gpickle('Sample DDI networks/net-ddi-indy-tau.gpickle')
Layers = {'BNU':G0,'CAT':G1,'IND':G2}

In [3]:
all_nodes = set([*list(G0.nodes()),*list(G1.nodes()),*list(G2.nodes())])

for node in all_nodes:
    for G in Layers.values():
        if node not in G.nodes():
            G.add_node(node)

set(Layers['BNU'].nodes())==set(Layers['CAT'].nodes())==set(Layers['IND'].nodes())

True

In [46]:
print(f'{len(all_nodes)=}')
print(f'{Layers["BNU"].number_of_edges()=}')
print(f'{Layers["CAT"].number_of_edges()=}')
print(f'{Layers["IND"].number_of_edges()=}')

all_edges = set()
for L,G in Layers.items():
    all_edges |= set(((u,L),(v,L)) for u,v in G.edges()) 
    all_edges |= set(((v,L),(u,L)) for u,v in G.edges())
total_edges = len(all_edges)//2
print(f'{total_edges=}')


len(all_nodes)=381
Layers["BNU"].number_of_edges()=146
Layers["CAT"].number_of_edges()=708
Layers["IND"].number_of_edges()=776
total_edges=1630


In [5]:
KG = kg.KnowledgeGraph(Layers)

In [6]:
REBUILD_CLOSURE = False
if REBUILD_CLOSURE:
    arglist = mp_for_nb.make_arglist(KG,'BNU',cut=7)

    with Pool() as pool:
        results = pool.imap_unordered(mp_for_nb.appd_cut,arglist)
        APPD = list(results)
        
    closure = {k[0]: {vk[0]: vv for vk, vv in v.items()} for k, v in APPD}
    pickle.dump(closure, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open("DDI_MultiClosure.pickle", "rb") as f:
        closure = pickle.load(f)

removed_edges_mp = KG.pareto_backbone_removed_edges(closure=closure)

In [7]:
BB0=dc.distance_closure(G0,only_backbone=True)
BB1=dc.distance_closure(G1,only_backbone=True)
BB2=dc.distance_closure(G2,only_backbone=True)

removed_edges_BB = []
for BB,L in zip( [BB0,BB1,BB2], ['BNU','CAT','IND']):
    for u,v,d in list(BB.edges(data=True)):
        if not d['is_trig']:
            removed_edges_BB.append(((u,L),(v,L)))
            BB.remove_edge(u,v)


In [8]:
sre = set(removed_edges_mp | set((v,u) for u,v in removed_edges_mp))
sreBB = set(removed_edges_BB+[(v,u) for u,v in removed_edges_BB])
print(sreBB-sre)
print(sre-sreBB)
print(len(sreBB)/2)
print(len(sre)/2)

set()
set()
53.0
53.0


In [9]:
LayersBB = {'BNU':BB0,'CAT':BB1,'IND':BB2}

In [10]:
all_nodesBB = set([*list(BB0.nodes()),*list(BB1.nodes()),*list(BB2.nodes())])

In [11]:
for node in all_nodesBB:
    for G in LayersBB.values():
        if node not in G.nodes():
            G.add_node(node)

set(LayersBB['BNU'].nodes())==set(LayersBB['CAT'].nodes())==set(LayersBB['IND'].nodes())

True

In [12]:
print(f'{len(all_nodesBB)=}')
print(f'{len(LayersBB["BNU"].edges())=}')
print(f'{len(LayersBB["CAT"].edges())=}')
print(f'{len(LayersBB["IND"].edges())=}')
print(f'{len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=}')


len(all_nodesBB)=381
len(LayersBB["BNU"].edges())=143
len(LayersBB["CAT"].edges())=660
len(LayersBB["IND"].edges())=774
len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=1577


In [57]:
layer_weights = {
    ('BNU','BNU'):1,
    ('CAT','CAT'):1,
    ('IND','IND'):1,
}
remove_edges_weighted = KG.weighted_backbone_removed_edges(
    closure = closure,
    layer_weights=layer_weights)

srew = set(remove_edges_weighted | set((v,u) for u,v in remove_edges_weighted))
print(f'kept edges (no weights): {total_edges-len(sre)//2}/{total_edges}')
print(f'kept edges (weighted layers): {total_edges-len(srew)//2}/{total_edges}')

kept edges (no weights): 1577/1630
kept edges (weighted layers): 1028/1630
